In [ ]:
import random

class Dice:
    def roll(self):
        return random.randint(1, 6)

class Player:
    def __init__(self, name):
        self.name = name
        self.scorecard = {}

    def choose_category(self):
        print(f"{self.name}, choose a category to score:")
        for category, value in self.scorecard.items():
            if value is None:
                print(f"{category}")
        category = input("Enter category: ").lower()
        while category not in self.scorecard or self.scorecard[category] is not None:
            print("Invalid category or already scored. Try again.")
            category = input("Enter category: ").lower()
        return category

class Yahtzee:
    def __init__(self, players):
        self.players = [Player(name) for name in players]
        self.dice = Dice()
        self.rounds = 13

    def play_round(self):
        for _ in range(3):
            rolls = [self.dice.roll() for _ in range(5)]
            print(f"Dice rolls: {rolls}")
            rerolls = input("Select dice to reroll (1-5) separated by spaces, or enter to keep all: ")
            if rerolls:
                reroll_indices = [int(x) - 1 for x in rerolls.split()]
                for index in reroll_indices:
                    rolls[index] = self.dice.roll()
                print(f"Dice rolls after reroll: {rolls}")
            category = self.current_player.choose_category()
            score = self.calculate_score(rolls, category)
            self.current_player.scorecard[category] = score
            print(f"{self.current_player.name} scores {score} points for {category}.\n")

    def calculate_score(self, rolls, category):
        if category == 'chance':
            return sum(rolls)
        elif category == 'yahtzee' and len(set(rolls)) == 1:
            return 50
        elif category in ('ones', 'twos', 'threes', 'fours', 'fives', 'sixes'):
            return rolls.count(int(category[-1])) * int(category[-1])
        elif category == 'three of a kind' and self.is_n_of_a_kind(rolls, 3):
            return sum(rolls)
        elif category == 'four of a kind' and self.is_n_of_a_kind(rolls, 4):
            return sum(rolls)
        elif category == 'full house' and self.is_full_house(rolls):
            return 25
        elif category == 'small straight' and self.is_small_straight(rolls):
            return 30
        elif category == 'large straight' and self.is_large_straight(rolls):
            return 40
        elif category == 'chance':
            return sum(rolls)
        else:
            return 0

    def is_n_of_a_kind(self, rolls, n):
        for num in set(rolls):
            if rolls.count(num) >= n:
                return True
        return False

    def is_full_house(self, rolls):
        return len(set(rolls)) == 2 and (rolls.count(rolls[0]) == 2 or rolls.count(rolls[0]) == 3)

    def is_small_straight(self, rolls):
        return len(set(rolls)) >= 4 and (max(rolls) - min(rolls)) >= 3

    def is_large_straight(self, rolls):
        return len(set(rolls)) == 5 and (max(rolls) - min(rolls)) == 4


    def print_scorecard(self):
        print("\nScorecard:")
        for player in self.players:
            print(f"{player.name}:")
            for category, score in player.scorecard.items():
                print(f"{category}: {score}")
            print()

    def run(self):
        print("Welcome to Yahtzee!")
        for _ in range(self.rounds):
            for player in self.players:
                self.current_player = player
                print(f"\n{player.name}'s turn:")
                self.play_round()
                self.print_scorecard()

if __name__ == "__main__":
    num_players = int(input("Enter number of players: "))
    player_names = [input(f"Enter name for player {i+1}: ") for i in range(num_players)]
    game = Yahtzee(player_names)
    game.run()

def test_calculate_score():
    yahtzee = Yahtzee(["Player1"])

    # Test for chance category
    assert yahtzee.calculate_score([1, 2, 3, 4, 5], 'chance') == 15
    assert yahtzee.calculate_score([6, 6, 6, 6, 6], 'chance') == 30

    # Test for yahtzee category
    assert yahtzee.calculate_score([1, 1, 1, 1, 1], 'yahtzee') == 50
    assert yahtzee.calculate_score([2, 2, 2, 2, 2], 'yahtzee') == 50
    assert yahtzee.calculate_score([1, 2, 3, 4, 5], 'yahtzee') == 0

    # Test for ones, twos, threes, fours, fives, and sixes categories
    assert yahtzee.calculate_score([1, 1, 2, 2, 3], 'ones') == 2
    assert yahtzee.calculate_score([1, 1, 2, 2, 3], 'twos') == 4
    assert yahtzee.calculate_score([1, 1, 2, 2, 3], 'threes') == 3
    assert yahtzee.calculate_score([4, 4, 4, 4, 5], 'fours') == 16
    assert yahtzee.calculate_score([4, 4, 4, 4, 5], 'fives') == 5
    assert yahtzee.calculate_score([6, 6, 6, 6, 6], 'sixes') == 30

    # Test for three of a kind and four of a kind categories
    assert yahtzee.calculate_score([1, 1, 1, 2, 3], 'three of a kind') == 8
    assert yahtzee.calculate_score([2, 2, 2, 2, 3], 'three of a kind') == 11
    assert yahtzee.calculate_score([1, 1, 1, 1, 1], 'three of a kind') == 5
    assert yahtzee.calculate_score([1, 1, 1, 2, 3], 'four of a kind') == 8
    assert yahtzee.calculate_score([2, 2, 2, 2, 3], 'four of a kind') == 9
    assert yahtzee.calculate_score([1, 1, 1, 1, 1], 'four of a kind') == 5

    # Test for full house category
    assert yahtzee.calculate_score([1, 1, 2, 2, 2], 'full house') == 25
    assert yahtzee.calculate_score([3, 3, 4, 4, 4], 'full house') == 25
    assert yahtzee.calculate_score([1, 1, 1, 1, 1], 'full house') == 0

    # Test for small straight category
    assert yahtzee.calculate_score([1, 2, 3, 4, 6], 'small straight') == 30
    assert yahtzee.calculate_score([1, 2, 3, 5, 6], 'small straight') == 30
    assert yahtzee.calculate_score([2, 3, 4, 5, 6], 'small straight') == 30
    assert yahtzee.calculate_score([1, 3, 4, 5, 6], 'small straight') == 0

    # Test for large straight category
    assert yahtzee.calculate_score([1, 2, 3, 4, 5], 'large straight') == 40
    assert yahtzee.calculate_score([2, 3, 4, 5, 6], 'large straight') == 40
    assert yahtzee.calculate_score([1, 2, 3, 4, 6], 'large straight') == 0

test_calculate_score()

Enter number of players: 1
Enter name for player 1: Pankhuri
Welcome to Yahtzee!

Pankhuri's turn:
Dice rolls: [6, 5, 1, 4, 2]
Select dice to reroll (1-5) separated by spaces, or enter to keep all: 1 2
Dice rolls after reroll: [6, 3, 1, 4, 2]
Pankhuri, choose a category to score:
Enter category: small straight
Invalid category or already scored. Try again.
